# Mathematics for AI - Coursework Task 2

The second task is about classifying handwritten digits. We will use the MNIST dataset for training and testing. The point of this task is to develop a multi-layer neural network for classificationusing mostly Numpy:

• Implement sigmoid and relu layers (with forward and backward pass)

• Implement a softmax output layer

• Implement a fully parameterizable neural network (number and types of layers, number of units)

• Implement an optimizer(e.g. SGD or Adam)and a stopping criterionof your choosing
• Train your Neural Network using backpropagation. Evaluate different neural network architectures andcompare your different results. You can also compare withthe results presented inhttp://yann.lecun.com/exdb/mnist/

In [5]:
# Import limited libraries
from sklearn.datasets import load_digits
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [33]:
# Load MNIST dataset

df = load_digits()

In [37]:
# Split dataset 
X = df.data
y = df.target

In [42]:
# Define network architecture 
nn_architecture = [
    {"input_dim": 2, "output_dim": 4, "activation": "relu"},
    {"input_dim": 4, "output_dim": 6, "activation": "relu"},
    {"input_dim": 6, "output_dim": 6, "activation": "relu"},
    {"input_dim": 6, "output_dim": 4, "activation": "relu"},
    {"input_dim": 4, "output_dim": 1, "activation": "softmax"},
]

https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795

In [17]:
# Mathematical functions

# Sigmoid function
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

# Derivative of Sigmoid (for backward propagation) 
def sigmoid_d(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

# ReLu function
def relu(Z):
    return np.maximum(0,Z)

# Derivative of ReLu
def relu_d(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

# Softmax function for output
def softmax(vector):
    e = exp(vector)
    return e / e.sum()

# Derivative of softmax
def softmax_d(dA, Z):
    soft = softmax(Z)
    return dA * soft * (1 - soft)

In [18]:
# Evaluation functions

# Cost function
def get_cost_value(Y_hat, Y):
    m = Y_hat.shape[1]
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

# Accuracy 
def get_accuracy_value(Y_hat, Y):
    Y_hat_ = convert_prob_into_class(Y_hat)
    return (Y_hat_ == Y).all(axis=0).mean()

In [43]:
# Initialize the network
def init_network(nn_architecture, seed = 9):
    np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_vals = {}

    for id, layer in enumerate(nn_architecture):
        layer_id = id + 1
        liz = layer["input"] # Set layer input size (liz)
        loz = layer["output"] # Set layer output size (loz)
    
        params_vals['W' + str(layer_id)] = np.random.randn(loz, liz) * 0.1 # Set random weight
        params_vals['b' + str(layer_id)] = np.random.randn(loz, 1) * 0.1 # Set random bias vector
            
    return params_vals

def single_layer_forward_propagagtion(self, A, W, b, activation):
    Z = np.dot(W, A) + b # Calculate Z
        
    # Logic to decide the activation function
    if activation == "relu":
        activation_function = relu
    elif activation == "sigmoid":
        activation_function = relu
    elif activation == "softmax":
        activation_function = relu
    else:
        raise Expection('Activation function not supported')

    return activation_func(Z), Z

def full_forward_propagation(X, params_vals, nn_architecture):
    mem = {}
    A_curr = X
        
    for id, layer in enumerate(nn_architecture):
        layer_id = id + 1
        A_prev = A_curr

        activ_function_curr = layer["activation"]
        W_curr = params_vals['W' + str(layer_id)]
        b_curr = params_vals ['b' + str(layer_id)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
            
        mem['A' + str(id)] = A_prev
        mem['Z' + str(layer_id)] = Z_curr

    return A_curr, mem

def single_layer_backward_propagation(dA, W, b, Z, A, activation):
    m = A.shape[1]

    if activation == 'relu':
        backward_activation_func = relu_d
    elif activation == 'sigmoid':
        backward_activation_func = sigmoid_d
    elif activation == 'softmax':
        backward_activation_func = softmax_d
    else:
        raise Expection('Activation function not supported')
        
    dZ_curr = backward_activation_func(dA, Z)
    dW_curr = np.dot(dZ_curr, A.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

def full_backward_propagation(Yh, Y, mem, params_vals, nn_architecture):
    grads_vals = {}
    m = Y.shape[1]
    Y = Y.reshape(Yh.shape)

    dA_prev = -(np.divide(Y, Yh) - np.divide(1 - Y, 1 - Yh))

    for layer_id_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_id_curr = layer_id_prev + 1
        activ_function_curr = layer['activation']
        aA_curr = dA_prev

        A_prev = memory["A" + str(layer_id_prev)]
        Z_curr = memory["Z" + str(layer_id_curr)]
        W_curr = params_vals["W" + str(layer_id_curr)]
        b_curr = params_vals["b" + str(layer_id_curr)]
        
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
        
        grads_vals["dW" + str(layer_id_curr)] = dW_curr
        grads_vals["db" + str(layer_id_curr)] = db_curr
    
    return grads_values
            
def update(params_vals, grads_vals, nn_architecture, learning_rate):
    for layer_id, layer in enumerate(nn_architecture):
        params_vals["W" + str(layer_id)] -= learning_rate * grads_vals["dW" + str(layer_id)]        
        params_vals["b" + str(layer_id)] -= learning_rate * grads_vals["db" + str(layer_id)]

    return params_vals;

In [44]:
def train(X, Y, nn_architecture, epochs, learning_rate):
    params_vals = init_layers(nn_architecture, 2)
    cost_history = []
    accuracy_history = []
    
    for i in range(epochs):
        Yh, cashe = full_forward_propagation(X, params_vals, nn_architecture)
        cost = get_cost_value(Yh, Y)
        cost_history.append(cost)
        accuracy = get_accuracy_value(Yh, Y)
        accuracy_history.append(accuracy)
        
        grads_vals = full_backward_propagation(Yh, Y, cashe, params_vals, nn_architecture)
        params_vals = update(params_vals, grads_vals, nn_architecture, learning_rate)
        
    return params_vals, cost_history, accuracy_history

In [45]:
train(X, y, nn_architecture, 100, 1)

ValueError: shapes (4,2) and (1797,64) not aligned: 2 (dim 1) != 1797 (dim 0)

In [ ]:
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
        dataset_split = list()
        dataset_copy = list(dataset)
        fold_size = int(len(dataset) / n_folds)
        for i in range(n_folds):
            fold = list()
            while len(fold) < fold_size:
                index = randrange(len(dataset_copy))
                fold.append(dataset_copy.pop(index))
            dataset_split.append(fold)
        return dataset_split
    
 # Calculate accuracy percentage
def accuracy_metric(actual, predicted):
        correct = 0
        for i in range(len(actual)):
            if actual[i] == predicted[i]:
                correct += 1
        return correct / float(len(actual)) * 100.0
    
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
        folds = cross_validation_split(dataset, n_folds)
        scores = list()
        for fold in folds:
            train_set = list(folds)
            train_set.remove(fold)
            train_set = sum(train_set, [])
            test_set = list()
            for row in fold:
                row_copy = list(row)
                test_set.append(row_copy)
                row_copy[-1] = None
            predicted = algorithm(train_set, test_set, *args)
            actual = [row[-1] for row in fold]
            accuracy = accuracy_metric(actual, predicted)
            scores.append(accuracy)
        return scores
    
 # Calculate neuron activation for an input
def activate(weights, inputs):
        activation = weights[-1]
        for i in range(len(weights)-1):
            activation += weights[i] * inputs[i]
        return activation
    
 # Transfer neuron activation
def transfer(activation):
        return 1.0 / (1.0 + exp(-activation))
    
 # Forward propagate input to a network output
def forward_propagate(network, row):
        inputs = row
        for layer in network:
            new_inputs = []
            for neuron in layer:
                activation = activate(neuron['weights'], inputs)
                neuron['output'] = transfer(activation)
                new_inputs.append(neuron['output'])
            inputs = new_inputs
        return inputs
    
 # Calculate the derivative of an neuron output
def transfer_derivative(output):
        return output * (1.0 - output)
    
 # Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
        for i in reversed(range(len(network))):
            layer = network[i]
            errors = list()
            if i != len(network)-1:
                for j in range(len(layer)):
                    error = 0.0
                    for neuron in network[i + 1]:
                        error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
            else:
                for j in range(len(layer)):
                    neuron = layer[j]
                    errors.append(expected[j] - neuron['output'])
            for j in range(len(layer)):
                neuron = layer[j]
                neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
                
 # Update network weights with error
def update_weights(network, row, l_rate):
        for i in range(len(network)):
            inputs = row[:-1]
            if i != 0:
                inputs = [neuron['output'] for neuron in network[i - 1]]
            for neuron in network[i]:
                for j in range(len(inputs)):
                    neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
                neuron['weights'][-1] += l_rate * neuron['delta']
                
 # Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
        for epoch in range(n_epoch):
            for row in train:
                outputs = forward_propagate(network, row)
                expected = [0 for i in range(n_outputs)]
                expected[row[-1]] = 1
                backward_propagate_error(network, expected)
                update_weights(network, row, l_rate)
                
 # Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
        network = list()
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
        network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
        network.append(output_layer)
        return network
    
 # Make a prediction with a network
def predict(network, row):
        outputs = forward_propagate(network, row)
        return outputs.index(max(outputs))
    
 # Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
        n_inputs = len(train[0]) - 1
        n_outputs = len(set([row[-1] for row in train]))
        network = initialize_network(n_inputs, n_hidden, n_outputs)
        train_network(network, train, l_rate, n_epoch, n_outputs)
        predictions = list()
        for row in test:
            prediction = predict(network, row)
            predictions.append(prediction)
        return(predictions)

In [ ]:
# Test Backprop on Seeds dataset
# evaluate algorithmn_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 2
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))